### **STEP-1**. Prepare for AI Solution Registration
- [x] **등록 할 AI Contents 의 experimental_plan.yaml 를 ./config/ 에 준비해 둡니다.**

- [x] **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     
<div style="margin-left: 20px">
```console
## ALO 의 main.py 위치에서 실행
conda create -n {name} python=3.10
conda init bash
conda activate {name}

python main.py 

pip install ipykernel
pip install requests 
python -m ipykernel install --user --name {name} --display-name [ipykernel-name]
```

- [x] **해당 jupyter notebook 에서 생성된 ipykernel 을 선택 합니다.**        
<div style="margin: 40px">
<img src="./image/ipykernel.png" width="300">
</div>

In [ ]:
import os
import json
import requests
from sm_control import SMC

<mark style="background: #FF5582A6;">TODO:</mark> pipeline.log 를 확인하는 방법 정리해 두기 

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 의 URI 입력
시스템 담담당에게 사용 가능한 URI 를 확인 합니다. 
- 고객지수플랫폼 Development: URI = "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경:       URI = "http://10.158.2.243:9999/"

In [ ]:
#################################
######     "Fixed ME"      ######
#################################
URI = "http://10.158.2.243:9999/"
WORKSPACE_NAME = "cism-ws"
#################################

In [ ]:
from IPython.display import HTML, display
import pandas as pd 


sol_name = requests.get(URI + 'api/v1/rass')
solution_list = []
for sols in sol_name.json()['rass']:
    solution_list.append(sols['name'])
    
## soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;">AI Solution 이름을 입력해 주세요!! </p>'))
user_input = input("Name: ")

# 기존 solution 존재하면 에러 나게 하기 
if user_input in solution_list: 
    txt = "Input 'AI Solution' exists. Raising an error."
    raise ValueError(HTML(f'<p style="font-size: 20px;">{txt}</p>'))
else:
    name = user_input ## 이름을 반영 합니다. 
    txt = f"It is possible to use the name '{user_input}' for AI Solution." 
    display(HTML(f'<p style="font-size: 14px;">{txt}</p>'))

print("\n")
df = pd.DataFrame(solution_list, columns=["AI Solution List"])
display(df)


#### **STEE-2-2**. Name 을 AI Conductor 에 등록합니다. 
: 이름이 등록되면 해당과정이 끝날 때, 까지 변경이 어려 울 수 있습니다. 변경이 필요할 경우 2-1) 를 다시 실행하여 주시기 바랍니다. 


In [ ]:
def print_styled(text, color='black', size='14px'):
    """
    Jupyter Notebook에서 스타일이 적용된 텍스트를 출력합니다.

    :param text: 출력할 텍스트
    :param color: 텍스트의 색상 (기본값: 검은색)
    :param size: 텍스트의 크기 (기본값: 14px)
    """
    display(HTML(f'<p style="color: {color}; font-size: {size};">{text}</p>'))

In [ ]:
### workspaces list 확인 
workspaces = requests.get(URI + 'api/v1/workspaces')

## ws_name 의 ECR, S3 를 확인 합니다. 
for ws in workspaces.json():
    if WORKSPACE_NAME in ws['name']:
        S3_BUCKET_NAME = ws['s3_bucket_name']
        ECR_NAME = ws['ecr_base_path']
        TAG = 'latest'        
        
txt = f"[INFO] S3_BUCUKET_URI:   '{S3_BUCKET_NAME}' "
print_styled(txt, 'green')

txt = f"[INFO] ECR_URI:   '{ECR_NAME}'"
print_styled(txt, 'green')


# solution metadata 를 setup 합니다.
SM_path = "./solution_meta_raw.yaml"
sm = SMC(workspaces, name)
sm.set_yaml()


#### **STEP-3**. Train 용 Sample Data 등록

In [ ]:
pipeline = "train"

# s3 접근확인
sm.s3_access_check()

# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
sm.s3_upload(pipeline)

#### **STEP-4**. Train Docker Container 제작

In [ ]:
#alo 제작을 위한 코드복사 (임시)
sm.set_docker_contatiner()

In [ ]:
#ecr 등록
# tags = [
#     "Key=Company,Value=LGE",
#     "Key=Owner,Value=IC360",
#     "Key=HQ,Value=CDO",
#     "Key=Division,Value=CDO",
#     "Key=Infra Region,Value=KIC",
#     "Key=Service Mode,Value=DE",
#     "Key=Cost Type,Value=COMPUTING",
#     "Key=Project,Value=CIS",
#     "Key=Sub Project,Value=CISM",
#     "Key=System,Value=AIDX"
# ]
# sm.set_aws_ecr(docker=False, tags=tags)
sm.set_aws_ecr(docker=True)

In [ ]:
# docker build 
sm.build_docker()

In [ ]:
sm.docker_push()

In [ ]:
sm.set_container_uri(pipeline) # uri도 그냥 입력되게 수정

#### **STEP-5**. Training 에 사용될 User Parameters 설정

In [ ]:
## user parameter 입력
sm.set_cadidate_param(pipeline)

## artifact 저장 경로 지정
sm.set_artifacts_uri(pipeline)

#### **STEP-6**. Training 에 사용될 Cloud resource 선택

In [ ]:
sm.set_resource(pipeline)

#### **STEP-7**. Inference 용 Sample Data 등록

In [ ]:
pipeline = "inference"

# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
sm.s3_upload(pipeline)

#### **STEP-8**. Inference 용 Docker Container 제작

In [ ]:
sm.set_docker_contatiner()

In [ ]:
#ecr 등록
# tags = [
#     "Key=Company,Value=LGE",
#     "Key=Owner,Value=IC360",
#     "Key=HQ,Value=CDO",
#     "Key=Division,Value=CDO",
#     "Key=Infra Region,Value=KIC",
#     "Key=Service Mode,Value=DE",
#     "Key=Cost Type,Value=COMPUTING",
#     "Key=Project,Value=CIS",
#     "Key=Sub Project,Value=CISM",
#     "Key=System,Value=AIDX"
# ]
# sm.set_aws_ecr(docker=False, tags=tags)
sm.set_aws_ecr(docker=True)

In [ ]:
# docker build 
sm.build_docker()


In [ ]:
sm.docker_push()

In [ ]:
sm.set_container_uri(pipeline) # uri도 그냥 입력되게 수정

#### **STEP-9**. Inference 용 User Parameters 제작

In [ ]:
sm.set_cadidate_param(pipeline)

sm.set_artifacts_uri(pipeline)
sm.set_model_uri(pipeline)

#### **STEP-10**. Inference 용 Resource 선택

In [ ]:
sm.set_resource(pipeline)

#### **STEP-11**. Description 추가

In [ ]:
sm.set_sm_description(name, "testing", f"without bucket url", f"without bucket url", "params", "alo", "s3://icon")


#### **STEP-12**. AI Solution 등록 !!

In [ ]:
sm.set_wrangler()

sm.set_edge()

# 등록을 위한 형태 변경
data = {
  "metadata_json": sm.sm_yaml
}

#json으로 변경
data =json.dumps(data)


#등록
post_response = requests.post(URI + 'api/v1/rass', data=data)
post_response_json = post_response.json()
post_response_json

rass_id = post_response_json['version']['id']

save_json = {"server_uri" : URI,
    "name" : name,
"version_id": rass_id,
"workspace_name": WORKSPACE_NAME}

# 조건문 넣기
import shutil
interface_dir = './interface'
try:
        # 폴더가 이미 존재하는 경우 삭제합니다.
    if os.path.exists(interface_dir):
        shutil.rmtree(interface_dir)
    
    # 새로운 폴더를 생성합니다.
    os.mkdir(interface_dir)
except Exception as e:
    print(f"폴더 생성 중 오류 발생: {e}")

with open(interface_dir + "/ai_soluition_certification.json", 'w') as outfile:
    json.dump(save_json, outfile)
